In [1]:
import numpy as np
from torch import tensor, no_grad, Tensor
import torch
import json

from thesis_v2.models.maskcnn_polished_with_rcnn_k_bl.builder import (
    gen_maskcnn_polished_with_rcnn_k_bl, load_modules
)

load_modules()

In [2]:
np.random.seed(0)
random_img = np.random.random_sample((10, 50, 50, 1))*2 - 1
random_img = random_img.astype(np.float32)
random_img = random_img.transpose((0,3,1,2))

In [3]:
from torchnetjson.builder import build_net

def make_model(
    *,
    n_timesteps,
    acc_mode,
    pooling_ksize,
    debug_args=None,
):
    model_json = gen_maskcnn_polished_with_rcnn_k_bl(
        (50, 50), 3,
        n_timesteps=n_timesteps,
        acc_mode=acc_mode,
        pooling_ksize=pooling_ksize,
        debug_args=debug_args,
        out_channel=48,
        kernel_size_l1=13,
        kernel_size_l23=3,
        factored_constraint=None,
        act_fn='softplus',
        pooling_type='max',
        num_layer=3,
        blstack_pool_ksize=1,
        blstack_pool_type=None,
        bn_after_fc=False,
    )
    print(json.dumps(model_json, indent=2))
    torch.manual_seed(0)
    model = build_net(model_json)
    print(model)
    with no_grad():
        for x, y in model.named_parameters():
            print(x, y.size(), y.mean(), y.std())

    return model




In [4]:
from torch.nn.functional import max_pool2d

In [5]:
from torch.backends import cudnn
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# let's put some test data.
def eval_model(model_to_eval, check=True, model_args=None):
    model_to_eval.cuda().eval()

    with no_grad():
        out_this = model_to_eval(tensor(random_img).cuda())
        if check:
            assert type(out_this) is tuple and len(out_this) == 1
    
    # let's do forward manually to make sure.
    assert model_args is not None
    if model_args is not None:
        n_timesteps = model_to_eval.get_module('bl_stack').n_timesteps
        with no_grad():
            # first, do the conv part.
            x = tensor(random_img).cuda()
            x = model_to_eval.get_module('bn_input')(x)
            x = model_to_eval.get_module('bl_stack')(x)
            # accumulator
            if model_args['acc_mode'] == 'instant':
                pass
            elif model_args['acc_mode'] == 'cummean':
                x = tuple(torch.mean(torch.stack(x[:i + 1]), 0) for i in range(len(x)))
            else:
                raise ValueError
            # second, the sequential part.
            only_fc = model_args['debug_args'].get('only_fc', False)
            if only_fc:
                assert model_args['pooling_ksize'] == 1
                x = tuple(model_to_eval.get_module('fc')(z) for z in x)
            else:
                # do pooling.
                x = tuple(max_pool2d(
                    z, kernel_size=model_args['pooling_ksize'],
                    ceil_mode=True) for z in x)
                x = tuple(model_to_eval.get_module('fc')(z) for z in x)
                if model_args.get('bn_output', False):
                    x = tuple(model_to_eval.get_module('bn_output')(z) for z in x)
                x = tuple(model_to_eval.get_module('final_act')(z) for z in x)
            
            if model_args['debug_args'].get('stack', True):
                x = (torch.stack(x, 0),)
            
            
        assert type(out_this) is type(x)
        if type(out_this) is Tensor:
            raise RuntimeError
        else:
            assert type(out_this) is tuple
            print(len(out_this), len(x))
            print(x[0].size())
            assert len(out_this) == len(x)
            for k1, k2 in zip(out_this, x):
                assert torch.equal(k1, k2)
    
    if check:
        print(out_this[0].size())
        return out_this[0].detach().cpu().numpy()
    else:
        return out_this


def check_ff():
    model_a = make_model(
        n_timesteps=1,
        acc_mode='instant',
        pooling_ksize=2,
    )
    model_b = make_model(
        n_timesteps=1,
        acc_mode='cummean',
        pooling_ksize=2,
    )
    out_a = eval_model(model_a,
                       model_args={
                           'acc_mode': 'instant',
                           'pooling_ksize': 2,
                           'debug_args': {},
                       }
                      )
    out_b = eval_model(model_b,
                       model_args={
                           'acc_mode': 'cummean',
                           'pooling_ksize': 2,
                           'debug_args': {},
                       }
                      )
    
    assert np.array_equal(out_a, out_b)

    
def check_instant_vs_cummean():
    model_a = make_model(
        n_timesteps=8,
        acc_mode='instant',
        pooling_ksize=1,
        debug_args={'only_fc': True}
    )
    model_b = make_model(
        n_timesteps=8,
        acc_mode='cummean',
        pooling_ksize=1,
        debug_args={'only_fc': True}
    )
    out_a = eval_model(model_a,
                       model_args={
                           'acc_mode': 'instant',
                           'debug_args': {'only_fc': True},
                           'pooling_ksize': 1,
                       }
                      )
    out_b = eval_model(model_b,
                       model_args={
                           'acc_mode': 'cummean',
                           'debug_args': {'only_fc': True},
                           'pooling_ksize': 1,
                       }
                      )
    
    assert out_a.shape == out_b.shape == (8, 10, 3)
    
    for idx in range(8):
        instant_out = np.mean(out_a[:idx+1], axis=0)
        cummean_out = out_b[idx]
        assert instant_out.shape == cummean_out.shape == (10,3)
        print(instant_out.mean(), instant_out.std())
        print(cummean_out.mean(), cummean_out.std())
        assert np.allclose(instant_out, cummean_out, atol=1e-5)

def check_stack_vs_no_stack():
    model_a = make_model(
        n_timesteps=8,
        acc_mode='cummean',
        pooling_ksize=2,
    )
    model_b = make_model(
        n_timesteps=8,
        acc_mode='cummean',
        pooling_ksize=2,
        debug_args={'stack': False}
    )
    
    out_a = eval_model(model_a, check=False,
                       model_args={
                           'acc_mode': 'cummean',
                           'debug_args': {},
                           'pooling_ksize': 2
                       }
                      )
    out_b = eval_model(model_b, check=False,
                       model_args={
                           'acc_mode': 'cummean',
                           'debug_args': {'stack': False},
                           'pooling_ksize': 2
                       }
                      )
    out_a = out_a[0].detach().cpu().numpy()
    out_b = [x.detach().cpu().numpy() for x in out_b]
    assert out_a.shape == (8, 10, 3)
    assert np.array_equal(out_a, np.asarray(out_b))

In [6]:
check_ff()

{'final_act', 'fc', 'pooling', 'bn_output'}
{
  "module_dict": {
    "bn_input": {
      "name": "torch.nn.batchnorm2d",
      "params": {
        "num_features": 1,
        "eps": 0.001,
        "momentum": 0.1,
        "affine": true
      },
      "init": {}
    },
    "bl_stack": {
      "name": "rcnn_kriegeskorte.blstack",
      "params": {
        "n_timesteps": 1,
        "channel_list": [
          1,
          48,
          48,
          48
        ],
        "ksize_list": [
          13,
          3,
          3
        ],
        "pool_ksize": 1,
        "pool_type": null,
        "act_fn": "softplus"
      },
      "init": {
        "strategy": "normal",
        "parameters": {
          "std": 0.01
        }
      }
    },
    "accumulator": {
      "name": "rcnn_kriegeskorte.accumulator",
      "params": {
        "mode": "instant"
      },
      "init": null
    },
    "pooling": {
      "name": "torch.nn.maxpool2d",
      "params": {
        "kernel_size": 2,
        "c

1 1
torch.Size([1, 10, 3])
torch.Size([1, 10, 3])
1 1
torch.Size([1, 10, 3])
torch.Size([1, 10, 3])


In [7]:
check_instant_vs_cummean()

{'fc'}
{
  "module_dict": {
    "bn_input": {
      "name": "torch.nn.batchnorm2d",
      "params": {
        "num_features": 1,
        "eps": 0.001,
        "momentum": 0.1,
        "affine": true
      },
      "init": {}
    },
    "bl_stack": {
      "name": "rcnn_kriegeskorte.blstack",
      "params": {
        "n_timesteps": 8,
        "channel_list": [
          1,
          48,
          48,
          48
        ],
        "ksize_list": [
          13,
          3,
          3
        ],
        "pool_ksize": 1,
        "pool_type": null,
        "act_fn": "softplus"
      },
      "init": {
        "strategy": "normal",
        "parameters": {
          "std": 0.01
        }
      }
    },
    "accumulator": {
      "name": "rcnn_kriegeskorte.accumulator",
      "params": {
        "mode": "instant"
      },
      "init": null
    },
    "pooling": {
      "name": "torch.nn.maxpool2d",
      "params": {
        "kernel_size": 1,
        "ceil_mode": true
      },
      "init"

1 1
torch.Size([8, 10, 3])
torch.Size([8, 10, 3])
0.014933402 0.040460978
0.014933402 0.040460978
0.013559388 0.04107457
0.013559388 0.04107457
0.013146769 0.041380286
0.01314677 0.04138029
0.0129326815 0.04153129
0.0129326815 0.041531295
0.012805243 0.04162415
0.012805242 0.041624155
0.012719977 0.04168675
0.012719978 0.04168675
0.012658913 0.041731376
0.012658912 0.041731384
0.0126131 0.041764945
0.012613096 0.041764945


In [8]:
check_stack_vs_no_stack()

{'final_act', 'fc', 'pooling', 'bn_output'}
{
  "module_dict": {
    "bn_input": {
      "name": "torch.nn.batchnorm2d",
      "params": {
        "num_features": 1,
        "eps": 0.001,
        "momentum": 0.1,
        "affine": true
      },
      "init": {}
    },
    "bl_stack": {
      "name": "rcnn_kriegeskorte.blstack",
      "params": {
        "n_timesteps": 8,
        "channel_list": [
          1,
          48,
          48,
          48
        ],
        "ksize_list": [
          13,
          3,
          3
        ],
        "pool_ksize": 1,
        "pool_type": null,
        "act_fn": "softplus"
      },
      "init": {
        "strategy": "normal",
        "parameters": {
          "std": 0.01
        }
      }
    },
    "accumulator": {
      "name": "rcnn_kriegeskorte.accumulator",
      "params": {
        "mode": "cummean"
      },
      "init": null
    },
    "pooling": {
      "name": "torch.nn.maxpool2d",
      "params": {
        "kernel_size": 2,
        "c

8 8
torch.Size([10, 3])
